
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [95]:
# import libraries
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from sklearn.impute import SimpleImputer

## Directory

In [96]:
def process_json_files(directory_path, output_csv_path):
    """
    Process JSON files in the specified directory, normalize the data, and save it to a CSV file.

    Parameters:
    - directory_path: str, path to the directory containing JSON files.
    - output_csv_path: str, path to save the output CSV file.

    Returns:
    - None
    """
    # List all JSON files in the specified directory
    files = [f for f in os.listdir(directory_path) if f.endswith('.json')]
    print("Files in directory:")
    print(files)

    # Initialize an empty list to store dataframes
    df_list = []

    # Iterate over each JSON file in the directory
    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # First level normalization
        df = pd.json_normalize(data['data']['results'])
        
        # Second level normalization
        if 'branding' in df.columns:
            branding = pd.json_normalize(df['branding'].explode())
            branding.columns = [f'branding.{col}' for col in branding.columns]
            df = df.drop('branding', axis=1).join(branding)

        if 'description' in df.columns:
            description = pd.json_normalize(df['description'])
            description.columns = [f'description.{col}' for col in description.columns]
            df = df.drop('description', axis=1).join(description)

        if 'flags' in df.columns:
            flags = pd.json_normalize(df['flags'])
            flags.columns = [f'flags.{col}' for col in flags.columns]
            df = df.drop('flags', axis=1).join(flags)

        if 'lead_attributes' in df.columns:
            lead_attributes = pd.json_normalize(df['lead_attributes'])
            lead_attributes.columns = [f'lead_attributes.{col}' for col in lead_attributes.columns]
            df = df.drop('lead_attributes', axis=1).join(lead_attributes)

        if 'location.address' in df.columns:
            location_address = pd.json_normalize(df['location.address'])
            location_address.columns = [f'location.address.{col}' for col in location_address.columns]
            df = df.drop('location.address', axis=1).join(location_address)

        if 'location.county' in df.columns:
            location_county = pd.json_normalize(df['location.county'])
            location_county.columns = [f'location.county.{col}' for col in location_county.columns]
            df = df.drop('location.county', axis=1).join(location_county)

        if 'products' in df.columns:
            products = pd.json_normalize(df['products'])
            products.columns = [f'products.{col}' for col in products.columns]
            df = df.drop('products', axis=1).join(products)
        
        # Third level normalization
        if 'location.address.coordinate' in df.columns:
            location_coordinates = pd.json_normalize(df['location.address.coordinate'])
            location_coordinates.columns = [f'location.address.coordinate.{col}' for col in location_coordinates.columns]
            df = df.drop('location.address.coordinate', axis=1).join(location_coordinates)

        if 'source.agents' in df.columns:
            source_agents = pd.json_normalize(df['source.agents'].explode())
            source_agents.columns = [f'source.agents.{col}' for col in source_agents.columns]
            df = df.drop('source.agents', axis=1).join(source_agents)

        if 'other_listings.rdc' in df.columns:
            other_listings_rdc = pd.json_normalize(df['other_listings.rdc'].explode())
            other_listings_rdc.columns = [f'other_listings.rdc.{col}' for col in other_listings_rdc.columns]
            df = df.drop('other_listings.rdc', axis=1).join(other_listings_rdc)
        
        # Append the dataframe to the list
        df_list.append(df)

    # Explicitly cast object-dtype columns with boolean values to boolean dtype
    for df in df_list:
        bool_cols = df.select_dtypes(include=['object']).columns
        for col in bool_cols:
            if df[col].dropna().apply(lambda x: isinstance(x, bool)).all():
                df[col] = df[col].astype(bool)

    # Combine all dataframes
    combined_df = pd.concat(df_list, ignore_index=True)

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(output_csv_path, index=False)

    print(f"Data saved to {output_csv_path}")

In [97]:
# example usage:
directory_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data'
output_csv_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data/processed_data.csv'
process_json_files(directory_path, output_csv_path)

Files in directory:
['AK_Juneau_0.json', 'AK_Juneau_1.json', 'AK_Juneau_2.json', 'AK_Juneau_3.json', 'AK_Juneau_4.json', 'AL_Montgomery_0.json', 'AL_Montgomery_1.json', 'AL_Montgomery_2.json', 'AL_Montgomery_3.json', 'AL_Montgomery_4.json', 'AR_LittleRock_0.json', 'AR_LittleRock_1.json', 'AR_LittleRock_2.json', 'AR_LittleRock_3.json', 'AR_LittleRock_4.json', 'AZ_Phoenix_0.json', 'AZ_Phoenix_1.json', 'AZ_Phoenix_2.json', 'AZ_Phoenix_3.json', 'AZ_Phoenix_4.json', 'CA_Sacramento_0.json', 'CA_Sacramento_1.json', 'CA_Sacramento_2.json', 'CA_Sacramento_3.json', 'CA_Sacramento_4.json', 'CO_Denver_0.json', 'CO_Denver_1.json', 'CO_Denver_2.json', 'CO_Denver_3.json', 'CO_Denver_4.json', 'CT_Hartford_0.json', 'CT_Hartford_1.json', 'CT_Hartford_2.json', 'CT_Hartford_3.json', 'CT_Hartford_4.json', 'DE_Dover_0.json', 'DE_Dover_1.json', 'DE_Dover_2.json', 'DE_Dover_3.json', 'DE_Dover_4.json', 'FL_Tallahassee_0.json', 'FL_Tallahassee_1.json', 'FL_Tallahassee_2.json', 'FL_Tallahassee_3.json', 'FL_Talla

At this point, ensure that you have all sales in a dataframe.

In [75]:
df = pd.read_csv(output_csv_path)

In [76]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

- Is each cell one value, or do some cells have lists?

In [77]:
df.head()

,last_update_date,tags,permalink,status,list_date,open_houses,list_price,property_id,photos,community,virtual_tours,listing_id,price_reduced_amount,matterport,primary_photo.href,source.plan_id,source.spec_id,source.type,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,description.baths_full,description.name,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.sub_type,description.baths_1qtr,description.garage,description.stories,description.beds,description.type,lead_attributes.show_contact_an_agent,flags.is_new_construction,flags.is_for_rent,flags.is_subdivision,flags.is_contingent,flags.is_price_reduced,flags.is_pending,flags.is_foreclosure,flags.is_plan,flags.is_coming_soon,flags.is_new_listing,products.brand_name,location.address.postal_code,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,other_listings,branding.name,branding.photo,branding.type,source.agents.office_name,other_listings.rdc.listing_id,other_listings.rdc.listing_key,other_listings.rdc.status,other_listings.rdc.primary,community.advertisers,community.description.name
0,2023-09-19T20:52:50Z,"['carport', 'community_outdoor_space', 'cul_de...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,False,554950.0,9.074431e+09,"[{'tags': [{'label': 'house_view', 'probabilit...",False,False,2957241843,45000.0,False,https://ap.rdcpix.com/07097d34c98a59ebb7996889...,NaN,NaN,mls,1963.0,False,2023-09-18,False,2.0,False,False,10454.0,1821.0,2.0,NaN,False,1.0,NaN,3.0,single_family,True,False,False,False,False,False,False,False,False,False,False,basic_opt_in,99801.0,Alaska,-134.59372,58.36395,Juneau,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,EXP Realty LLC - Southeast Alaska,False,Office,EXP Realty LLC - Southeast Alaska,2.957242e+09,NaN,sold,True,NaN,NaN
1,NaN,NaN,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,NaN,False,NaN,9.424984e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-22,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,Non-Member Office,2.957023e+09,NaN,off_market,False,NaN,NaN
2,NaN,NaN,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,NaN,False,NaN,9.479069e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-22,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958935e+09,NaN,sold,False,NaN,NaN
3,NaN,NaN,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,NaN,False,NaN,9.879332e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-21,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,17850 Point Stephens Rd,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958935e+09,NaN,sold,False,NaN,NaN
4,NaN,NaN,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,NaN,False,NaN,9.521640e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-21,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,9951 Stephen Richards Memorial Dr,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958925e+

- What are the data types of each column?

In [78]:
# display all columns and their data types
print("Columns and their data types:")
print(df.dtypes)

Columns and their data types:
last_update_date                          object
tags                                      object
permalink                                 object
status                                    object
list_date                                 object
open_houses                               object
list_price                                object
property_id                              float64
photos                                    object
community                                 object
virtual_tours                             object
listing_id                                object
price_reduced_amount                      object
matterport                                object
primary_photo.href                        object
source.plan_id                            object
source.spec_id                            object
source.type                               object
description.year_built                    object
description.baths_3qtr                 

- Some sales may not actually include the sale price. These rows should be dropped.

In [79]:
# number of rows before dropping
rows_before = len(df)

# drop rows without 'description.sold_price' (target variable)
df = df.dropna(subset=['description.sold_price'])

# number of rows after dropping
rows_after = len(df)

# calculate the number of rows dropped
rows_dropped = rows_before - rows_after

# print the number of rows dropped
print(f"Number of rows dropped: {rows_dropped}")

Number of rows dropped: 37


- There are a lot of None values.  Should these be dropped or replaced with something?

In [80]:
# calculate the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100

# define a threshold for dropping columns (e.g., rule of thumb: more than 50% missing values)
threshold = 50

# identify columns to drop based on the threshold
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# drop the identified columns from the DataFrame
df = df.drop(columns=columns_to_drop)

# print the dropped columns
print(f"Dropped columns: {columns_to_drop}")

# display the remaining columns
print("Remaining columns after dropping:")
print(df.columns)

Dropped columns: Index(['virtual_tours', 'price_reduced_amount', 'description.baths_half',
       'description.sub_type', 'primary_photo', 'source', 'other_listings',
       'branding.photo', 'source.agents.office_name', 'community.advertisers',
       'community.description.name'],
      dtype='object')
Remaining columns after dropping:
Index(['last_update_date', 'tags', 'permalink', 'status', 'list_date',
       'open_houses', 'list_price', 'property_id', 'photos', 'community',
       'listing_id', 'matterport', 'primary_photo.href', 'source.plan_id',
       'source.spec_id', 'source.type', 'description.year_built',
       'description.baths_3qtr', 'description.sold_date',
       'description.sold_price', 'description.baths_full', 'description.name',
       'description.lot_sqft', 'description.sqft', 'description.baths',
       'description.baths_1qtr', 'description.garage', 'description.stories',
       'description.beds', 'description.type',
       'lead_attributes.show_contact_an_

In [82]:
# list of irrelevant columns to drop
columns_to_drop = ['permalink', 'photos', 'primary_photo.href', 'location.street_view_url']

# drop the specified columns
df = df.drop(columns=columns_to_drop)

- Some sales don't include the property type.

In [84]:
# impute missing values for categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer.fit_transform(df[categorical_cols])

In [85]:
# impute missing values for numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns
imputer = SimpleImputer(strategy='median')
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

In [86]:
# check the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_percentage)

Percentage of missing values in each column:
last_update_date                         0.0
tags                                     0.0
status                                   0.0
list_date                                0.0
open_houses                              0.0
list_price                               0.0
property_id                              0.0
community                                0.0
listing_id                               0.0
matterport                               0.0
source.plan_id                           0.0
source.spec_id                           0.0
source.type                              0.0
description.year_built                   0.0
description.baths_3qtr                   0.0
description.sold_date                    0.0
description.sold_price                   0.0
description.baths_full                   0.0
description.name                         0.0
description.lot_sqft                     0.0
description.sqft                         0.0
descriptio

In [90]:
# function to cap outliers at a specified percentile
def cap_outliers(df, cols, lower_percentile=0.05, upper_percentile=0.95):
    for col in cols:
        lower_bound = df[col].quantile(lower_percentile)
        upper_bound = df[col].quantile(upper_percentile)
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

# identify all numerical columns to cap outliers
numerical_cols_to_cap = df.select_dtypes(include=['number']).columns

# cap outliers in these numerical columns
df = cap_outliers(df, numerical_cols_to_cap)

In [91]:
# inspect unique values in categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"Unique values in column '{col}':")
    print(df[col].value_counts())

Unique values in column 'last_update_date':
2024-01-12    862
2024-01-16    605
2024-01-11    560
2024-01-10    540
2024-01-05    367
2024-01-08    342
2023-12-29    284
2024-01-09    247
2024-01-04    227
2024-01-03    222
2024-01-15    203
2024-01-02    195
2024-01-13    190
2023-12-28    164
2023-12-19    158
2023-12-18    140
2023-12-22    136
2023-10-20    134
2024-01-06    128
2023-12-26    115
2023-12-06    108
2023-12-23    105
2024-01-14     99
2023-12-27     98
2023-12-21     90
2023-12-20     90
2023-12-15     71
2024-01-07     69
2023-12-30     65
2023-11-30     60
2023-12-08     58
2023-12-14     48
2023-12-01     48
2024-01-17     39
2023-12-31     37
2023-08-23     35
2023-12-13     35
2023-11-02     35
2023-11-21     30
2023-10-31     30
2023-12-05     29
2023-12-07     29
2023-12-04     26
2023-07-28     25
2023-12-11     25
2023-12-09     25
2023-11-14     25
2023-11-07     24
2023-12-16     23
2023-11-03     22
2023-11-13     21
2023-09-12     20
2023-10-24     20
20

In [92]:
# function to remove timezone information
def remove_timezone(dt):
    if dt.tzinfo is not None:
        return dt.replace(tzinfo=None)
    return dt

# ensure the columns are in datetime format
df['last_update_date'] = pd.to_datetime(df['last_update_date'], errors='coerce')
df['list_date'] = pd.to_datetime(df['list_date'], errors='coerce')
df['description.sold_date'] = pd.to_datetime(df['description.sold_date'], errors='coerce')

# remove timezone information if present
df['last_update_date'] = df['last_update_date'].apply(lambda x: remove_timezone(x) if pd.notnull(x) else x)
df['list_date'] = df['list_date'].apply(lambda x: remove_timezone(x) if pd.notnull(x) else x)
df['description.sold_date'] = df['description.sold_date'].apply(lambda x: remove_timezone(x) if pd.notnull(x) else x)

# format the dates to 'yyyy-mm-dd'
df['last_update_date'] = df['last_update_date'].dt.strftime('%Y-%m-%d')
df['list_date'] = df['list_date'].dt.strftime('%Y-%m-%d')
df['description.sold_date'] = df['description.sold_date'].dt.strftime('%Y-%m-%d')

# print the first few rows to verify the changes
print(df[['last_update_date', 'list_date', 'description.sold_date']].head())

  last_update_date   list_date description.sold_date
0       2023-09-19  2023-06-29            2023-09-18
1       2023-10-20  2024-01-05            2023-08-22
2       2023-10-20  2024-01-05            2023-08-22
3       2023-10-20  2024-01-05            2023-08-21
4       2023-10-20  2024-01-05            2023-08-21


In [93]:
# convert 'list_price' and 'description.sold_price' to numeric data types
df['list_price'] = pd.to_numeric(df['list_price'], errors='coerce')
df['description.sold_price'] = pd.to_numeric(df['description.sold_price'], errors='coerce')

# format the numeric values to two decimal places (if needed)
df['list_price'] = df['list_price'].map(lambda x: '{:.2f}'.format(x) if pd.notnull(x) else x)
df['description.sold_price'] = df['description.sold_price'].map(lambda x: '{:.2f}'.format(x) if pd.notnull(x) else x)

# print the first few rows to verify the changes
print(df[['list_price', 'description.sold_price']].head())

  list_price description.sold_price
0  554950.00                    NaN
1  450000.00                    NaN
2  450000.00                    NaN
3  450000.00                    NaN
4  450000.00                    NaN


- Maybe the "tags" will help create some features.

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized